In [2]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 

In [3]:
import ee
import geemap

In [4]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=2acWBMzzJZGrPz6mPJYQ0rA34hrS4HVJ665kwJRfhc4&tc=B27-rpRbsR_gVA8iP9EwMREg8olaugaa7BiD4QhXpSU&cc=Sx5WrEmI7tr_0cF2MYP1AyWYgpojROOgktyyLmIo2W4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7f83pcqH5lImVK4-bWqk6HTPVQgE2ZgZR4SxMR4rZtF8GZFgGLdVs

Successfully saved authorization token.


In [5]:
geometry =ee.Geometry.Polygon(
        [[[77.39980020154476, 23.209794968246],
          [77.40042247403622, 23.209794968246],
          [77.4008516274786, 23.209755526121455],
          [77.40121640790463, 23.209479430923615],
          [77.40158118833065, 23.209696362912805],
          [77.40143098462582, 23.210011899722915],
          [77.40098037351132, 23.210189388851145],
          [77.40035810101986, 23.210149946843007],
          [77.40115203488827, 23.210603529233456],
          [77.40093745816708, 23.210938784793257],
          [77.40100183118344, 23.21139236450578],
          [77.4014095269537, 23.211767059628617],
          [77.40100183118344, 23.211688176532185],
          [77.40025081265927, 23.211806501159376],
          [77.40014352429867, 23.21147124777687],
          [77.39992894757748, 23.21115571441311],
          [77.39962854016781, 23.210781017575787],
          [77.39945687879086, 23.210386598717097],
          [77.39930667508602, 23.20989357350644]]]);

In [6]:
import numpy as np
import pandas as pd
start_date = '2020-01-01'
end_date = '2021-12-31'

def get_data(start_date, end_date):
    bands = ['B3','B8']
    dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(geometry) \
                    .filterDate(start_date,end_date) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                    .median() 
    
    ndwi = dataset.normalizedDifference(bands).rename('ndwi')
    latlon = ee.Image.pixelLonLat().addBands(ndwi)
    latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=geometry,
          scale=30)
    data_ndwi = np.array((ee.Array(latlon.get('ndwi')).getInfo()))

    band_ndci = ['B4','B3']
    band_do=['B4','B5']
    band_temp=['B10']
    band_ndti = ['B8','B12']
    band_ndsi = ['B3','B11']
    dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                      .filterBounds(geometry) \
                      .filterDate(start_date,end_date) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                      .median() 
    
    ndci = dataset.normalizedDifference(bands).rename('ndci')
    latlon = ee.Image.pixelLonLat().addBands(ndci)
    latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=geometry,
          scale=30)
    assert len(band_ndci) == 2, 'bands_ndci should contain exactly 2 band names'
    data_ndci = np.array((ee.Array(latlon.get('ndci')).getInfo()))

    dissolveoxygen = dataset.normalizedDifference(band_do).rename('dissolveoxygen')
    latlon = ee.Image.pixelLonLat().addBands(dissolveoxygen)
    latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=geometry,
          scale=30,
          tileScale = 16)
    data_do = np.array((ee.Array(latlon.get('dissolveoxygen')).getInfo()))

    temp = dataset.select(band_temp).rename('temp')
    latlon = ee.Image.pixelLonLat().addBands(temp)
    latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=geometry,
            scale=30)
    ndti = dataset.normalizedDifference(bands).rename('ndti')
    latlon = ee.Image.pixelLonLat().addBands(ndti)
    latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=geometry,
          scale=30)
    data_ndti = np.array((ee.Array(latlon.get('ndti')).getInfo()))

    ndsi=dataset.normalizedDifference(bands).rename('ndsi')
    latlon = ee.Image.pixelLonLat().addBands(ndsi)
    # apply reducer to list
    latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=geometry,
          scale=30)
    data_ndsi = np.array((ee.Array(latlon.get('ndsi')).getInfo()))

    data = pd.DataFrame({'NDWI': data_ndwi,
                         'NDCI': data_ndci,
                         'NDTI':data_ndti,
                         'DO':data_do,
                         'NDSI':data_ndsi})
    return data
data = get_data(start_date, end_date)
print(data)
print(len(data))

        NDWI      NDCI      NDTI        DO      NDSI
0  -0.255780 -0.255780 -0.255780 -0.150908 -0.255780
1  -0.367697 -0.367697 -0.367697 -0.146540 -0.367697
2  -0.422582 -0.422582 -0.422582 -0.134282 -0.422582
3  -0.415623 -0.415623 -0.415623 -0.117402 -0.415623
4  -0.388830 -0.388830 -0.388830 -0.129893 -0.388830
5  -0.388381 -0.388381 -0.388381 -0.105805 -0.388381
6  -0.171976 -0.171976 -0.171976 -0.127626 -0.171976
7  -0.110311 -0.110311 -0.110311 -0.113475 -0.110311
8  -0.457541 -0.457541 -0.457541 -0.161295 -0.457541
9  -0.287702 -0.287702 -0.287702 -0.104331 -0.287702
10 -0.430798 -0.430798 -0.430798 -0.143835 -0.430798
11 -0.362693 -0.362693 -0.362693 -0.149101 -0.362693
12 -0.432718 -0.432718 -0.432718 -0.155708 -0.432718
13 -0.436235 -0.436235 -0.436235 -0.156131 -0.436235
14 -0.407311 -0.407311 -0.407311 -0.173748 -0.407311
15 -0.465861 -0.465861 -0.465861 -0.181047 -0.465861
16 -0.280122 -0.280122 -0.280122 -0.110057 -0.280122
17 -0.407474 -0.407474 -0.407474 -0.121588 -0.

In [7]:
from datetime import datetime, timedelta

def get_monthly_date_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    dates = []
    while start <= end:
        next_month = start.replace(day=28) + timedelta(days=4) # next month's 1st date
        end_month = next_month - timedelta(days=1) # current month's last date
        if end_month > end:
            end_month = end
        dates.append((start.strftime("%Y-%m-%d"), end_month.strftime("%Y-%m-%d")))
        start = next_month
    return dates

dates = get_monthly_date_range(start_date, end_date)

In [8]:
import pandas as pd

df_data = pd.DataFrame()
df_data["NDWI"] = []
df_data['NDCI'] = []
df_data['DO'] = []
df_data['NDTI'] = []
df_data['NDSI'] = []
df_data["time"] = []

for date in dates:
    try:
        our_data = get_data(date[0], date[1])
        mean_data = pd.DataFrame({
            'NDWI': our_data['NDWI'].mean(),
            'NDCI': our_data['NDCI'].mean(),
            'DO': our_data['DO'].mean(),
            'NDTI': our_data['NDTI'].mean(),
            'NDSI': our_data['NDSI'].mean(),
            'time': [date[0][0:7]]
        })
        df_data = pd.concat([df_data, mean_data], axis=0)
    except:
        pass

In [9]:
df_data

,NDWI,NDCI,DO,NDTI,NDSI,time
0,-0.413185,-0.413185,-0.190560,-0.413185,-0.413185,2020-01
0,-0.381174,-0.381174,-0.177902,-0.381174,-0.381174,2020-02
0,-0.555006,-0.555006,-0.265755,-0.555006,-0.555006,2020-03
0,-0.367539,-0.367539,-0.149386,-0.367539,-0.367539,2020-04
0,-0.393180,-0.393180,-0.161848,-0.393180,-0.393180,2020-05
0,-0.268536,-0.268536,-0.130584,-0.268536,-0.268536,2020-06
0,-0.483956,-0.483956,-0.277994,-0.483956,-0.483956,2020-09
0,-0.407065,-0.407065,-0.191150,-0.407065,-0.407065,2020-10
0,-0.391193,-0.391193,-0.164740,-0.391193,-0.391193,2020-11
0,-0.418462,-0.418462,-0.166574,-0.418462,-0.418462,2020-12


In [10]:
import plotly.express as px
import pandas as pd

# Create a sample dataframe
df = df_data

# Convert the 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Create separate line plots for each parameter
fig1 = px.line(df, x='time', y='NDWI', title='NDWI vs Time')
fig2 = px.line(df, x='time', y='NDCI', title='NDCI vs Time')
fig3 = px.line(df, x='time', y='DO', title='DO vs Time')
fig4 = px.line(df, x='time', y='NDTI', title='NDTI vs Time')
fig5 = px.line(df, x='time', y='NDSI', title='NDSI vs Time')

# Show the plots
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()


In [11]:
import plotly.express as px
import pandas as pd

# Create a sample dataframe
df = df_data

# Create a scatter plot with NDWI and NDCI as x and y axis
fig = px.scatter(df, x='NDWI', y='NDCI', title='NDWI vs NDCI')
fig2 = px.scatter(df, x='DO', y='NDTI',title = 'DO vs NDTI' )

# Show the plot
fig.show()
fig2.show()
